In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys; sys.path.append('../')
import pandas as pd
import tqdm
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import socceraction.vaep as vaep

In [2]:
## Configure file and folder names
datafolder = "../data"
spadl_h5 = os.path.join(datafolder,"spadl-statsbomb.h5")
predictions_h5 = os.path.join(datafolder,"predictions.h5")

In [3]:
games = pd.read_hdf(spadl_h5,"games")
games = games[games.competition_name == "FIFA World Cup"]
print("nb of games:", len(games))

nb of games: 64


In [4]:
players = pd.read_hdf(spadl_h5,"players")
teams = pd.read_hdf(spadl_h5,"teams")
actiontypes = pd.read_hdf(spadl_h5, "actiontypes")
bodyparts = pd.read_hdf(spadl_h5, "bodyparts")
results = pd.read_hdf(spadl_h5, "results")

A = []
for game in tqdm.tqdm(list(games.itertuples())):
    actions = pd.read_hdf(spadl_h5,f"actions/game_{game.game_id}")
    actions = (
        actions.merge(actiontypes)
        .merge(results)
        .merge(bodyparts)
        .merge(players,"left",on="player_id")
        .merge(teams,"left",on="team_id")
        .sort_values(["period_id", "time_seconds", "timestamp"])
        .reset_index(drop=True)
    )
    preds = pd.read_hdf(predictions_h5,f"game_{game.game_id}")
    values = vaep.value(actions,preds.scores,preds.concedes)
    A.append(pd.concat([actions,preds,values],axis=1))
A = pd.concat(A).sort_values(["game_id","period_id", "time_seconds", "timestamp"]).reset_index(drop=True)
A.columns

100%|██████████| 64/64 [00:05<00:00, 12.68it/s]


Index(['game_id', 'period_id', 'time_seconds', 'timestamp', 'team_id',
       'player_id', 'start_x', 'start_y', 'end_x', 'end_y', 'type_id',
       'result_id', 'bodypart_id', 'action_id', 'type_name', 'result_name',
       'bodypart_name', 'country_id', 'country_name', 'extra', 'jersey_number',
       'player_name', 'player_nickname', 'team_name', 'scores', 'concedes',
       'offensive_value', 'defensive_value', 'vaep_value'],
      dtype='object')

In [5]:
A["player"] = A[["player_nickname","player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)

A["count"] = 1

A_ = A[A.type_name != "shot_penalty"] # ignore penalties
players = (
    A_[["player_id","team_name","player","vaep_value","count"]]
    .groupby(["player_id","team_name","player"])
    .sum()
    .reset_index()
)
players = players.sort_values("vaep_value",ascending=False)
players

,player_id,team_name,player,vaep_value,count
281,5503,Argentina,Lionel Messi,3.939513,524
248,5470,Croatia,Ivan Rakitić,3.883688,835
593,10955,England,Harry Kane,3.726958,324
50,3308,England,Kieran Trippier,3.649573,689
122,4320,Brazil,Neymar,3.618000,749
44,3289,Belgium,Romelu Lukaku,3.553801,251
352,5574,Germany,Toni Kroos,3.459970,647
525,6196,Colombia,Yerry Mina,3.294680,325
265,5487,France,Antoine Griezmann,3.028749,546
121,4319,Uruguay,Edinson Cavani,2.891171,226


In [6]:
pg = pd.read_hdf(spadl_h5,"player_games")
pg = pg[pg.game_id.isin(games.game_id)]
mp = pg[["player_id","minutes_played"]].groupby("player_id").sum().reset_index()
stats = players.merge(mp)
stats = stats[stats.minutes_played > 150]
stats["vaep_rating"] = stats.vaep_value * 90 / stats.minutes_played
stats.sort_values("vaep_value",ascending=False)

,player_id,team_name,player,vaep_value,count,minutes_played,vaep_rating
0,5503,Argentina,Lionel Messi,3.939513,524,378,0.937979
1,5470,Croatia,Ivan Rakitić,3.883688,835,677,0.516295
2,10955,England,Harry Kane,3.726958,324,595,0.563742
3,3308,England,Kieran Trippier,3.649573,689,602,0.545617
4,4320,Brazil,Neymar,3.618000,749,478,0.681213
5,3289,Belgium,Romelu Lukaku,3.553801,251,488,0.655414
6,5574,Germany,Toni Kroos,3.459970,647,289,1.077499
7,6196,Colombia,Yerry Mina,3.294680,325,315,0.941337
8,5487,France,Antoine Griezmann,3.028749,546,580,0.469978
9,4319,Uruguay,Edinson Cavani,2.891171,226,355,0.732973
